In [177]:
import sys
import xarray as xr
import random
import numpy as np
import importlib as imp
import pandas as pd
import matplotlib.pyplot as plt
import pickle
#import matplotlib.colors as mcolors

%load_ext autoreload
%autoreload 2
import utils
import databuilder.filemethods as filemethods
from databuilder.data_generator import ClimateData
import databuilder.data_generator as data_generator
import databuilder.filemethods as filemethods

print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
python version = 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
numpy version = 1.26.4
xarray version = 2024.2.0


In [178]:
config = utils.get_config("exp000")
seed = config["seed_list"][0]

imp.reload(utils)
imp.reload(filemethods)
imp.reload(data_generator)

data = ClimateData(
    config["databuilder"], 
    expname = config["expname"],
    seed=seed,
    data_dir = config["data_dir"], 
    figure_dir=config["figure_dir"],
    fetch=True,
    verbose=False
)

f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')


IndexError: dimension coordinate 'time' conflicts between indexed and indexing objects:
<xarray.DataArray 'time' (time: 3949)> Size: 32kB
array([cftime.DatetimeNoLeap(1850, 1, 7, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 1, 8, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 1, 9, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(1860, 12, 29, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1860, 12, 30, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1860, 12, 31, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 32kB 1850-01-07 00:00:00 ... 1860-12-31 00:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T
vs.
<xarray.IndexVariable 'time' (time: 4015)> Size: 32kB
array([cftime.DatetimeNoLeap(1850, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 1, 2, 0, 0, 0, 0

In [ ]:
d_train, d_val, d_test = data.fetch_data()

f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
 da dims after processing data: ('time', 'lat', 'lon', 'channel')
f_dict[x] dims: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
 da dims after processing data: ('time', 'lat', 'lon', 'channel')
f_dict[x] dims: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop: ('time', 'lat', 'lon', 'channel')
f_dict[key] dims at end of loop

In [ ]:
d_train['x'].sel(channel = 1)

<xarray.DataArray 'SAMPLES' (time: 4015, lat: 30, lon: 260)> Size: 125MB
array([[[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]],

       [[            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
...
         -3.84493770e-09, -3.17518056e-09, -2.09328599e-09],
        [ 3.97684496e-10,  6.22743523e-10,  5.34595479e-10, ...,
         -2.94495739e-09, -4.37183223e-09, -5.97874861e-09],
        [ 2.23161684e-10,  2.28547639e-10,  4.40897249e-11, ...,
         -1.86145666e-09, -3.39735151e-09, -4.28341007e-09]],

       [[-1.50625432e-08,  2.80908741e-09, -4.60900340e-09, ...,
         -3.80291976e-09,  7.17509607e-10,  5.06973263e-09],
        [-7.51287654e-09,  6.11435746e-09, -2.24138530e-09, ...,
          1.31026723e-09,  4.49311299e-09,  8.40411207e-09],
        [-2.81788176e-10,  7.00891434e-09,  1.81623172e-09, ...,
          9.15203469e-09,  9.95038896e-09,  7.35916661e-09],
        ...,
        [ 5.42122770e-11,  6.86244334e-12,  6.51567550e-11, ...,
         -4.66377648e-09, -4.05422140e-09, -2.67847722e-09],
        [ 2.60703736e-10,  3.68706621e-10,  2.66402594e-10, ...,
         -3.28038863e-09, -4.23248459e-09, -5.22797805e-09],
        [ 3.11792647e-10,  3.26343980e-10,  2.01034717e-10, ...,
         -1.89002902e-09, -3.17143800e-09, -3.91238908e-09]]],
      dtype=float32)
Coordinates:
  * time     (time) object 32kB 1850-01-01 00:00:00 ... 1860-12-31 00:00:00
  * lon      (lon) float64 2kB 40.5 41.5 42.5 43.5 ... 296.5 297.5 298.5 299.5
  * lat      (lat) float64 240B -14.5 -13.5 -12.5 -11.5 ... 11.5 12.5 13.5 14.5
Attributes:
    long_name:     None
    units:         None
    cell_methods:  None

### MJO Composites: 

In [ ]:
from databuilder import indicescompositor

for 
MJOoutput = indicescompositor(000, )

SyntaxError: invalid syntax (3785567543.py, line 3)

/System/Volumes/Data/Users/C830793391/miniconda3/pkgs/libtiff-4.5.1-h313beb8_0/lib/libtiff.5.dylib


/System/Volumes/Data/Users/C830793391/miniconda3/envs/jvp1/lib/libtiff.5.dylib

Users/C830793391/miniconda3/envs/env-torch/lib/libtiff.6.dylib




In [ ]:
# import gzip, pickle

# data_savename = "/Users/eabarnes/big_data/e3sm_v2/presaved_data/exp001_train.pkl"
# with gzip.open(data_savename, "wb") as fp:
#     pickle.dump(x_train, fp)
#     pickle.dump(labels_train, fp)

#     pickle.dump(x_mean, fp)
#     pickle.dump(x_std, fp)
    
#     pickle.dump(lat, fp)
#     pickle.dump(lon, fp)

# data_savename = "/Users/eabarnes/big_data/e3sm_v2/presaved_data/exp001_val_test.pkl"
# with gzip.open(data_savename, "wb") as fp:
#     pickle.dump(x_val, fp)
#     pickle.dump(labels_val, fp)

#     pickle.dump(x_test, fp)
#     pickle.dump(labels_test, fp)
    
#     pickle.dump(lat, fp)
#     pickle.dump(lon, fp)